In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import lxml.etree as ET
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tiktoken

# ***Load and build dataset***

In [2]:
tokenizer = tiktoken.get_encoding('gpt2')
SEED = 42
BATCH_SIZE = 512
BLOCK_SIZE = 512

In [3]:
def get_and_transform_dataset():
    file_path = 'dataset/french-discussion-reddit/final_SPF_2.xml'
    # Initializes the parser
    parser = ET.XMLParser(recover=True)
    # Parses the file
    tree = ET.parse(file_path, parser=parser)
    xroot = tree.getroot()
    # One conversation -> one line in the data array
    dfcols = ['link_id', 'subreddit_id', 'uid',"comment_id",'score', 'parent_id', 'create_utc', 'text']
    data=np.array(([[ [node.attrib.get('link_id'),node.attrib.get('subreddit_id'), node.getchildren()[j].get('uid'), node.getchildren()[j].get('comment_id'), node.getchildren()[j].get('score'), node.getchildren()[j].get('parent_id'), node.getchildren()[j].get('create_utc'),node.getchildren()[j].text] for j in range(len(node.getchildren()))] for node in xroot]), dtype=object)
    print('number of conversations: ',data.shape[0])
    #one comments -> one line in the data array
    data=np.array([liste for conversation in data for liste in conversation], dtype=object)
    print('number of comments: ',data.shape[0])
    X = pd.DataFrame(data=data, columns=dfcols)["text"]
    X = np.array(X.values)
    print(X.shape)
    return X

def get_chunks(data,block_size=8):
    values = []
    for _,tokens in enumerate(data) :
        if len(tokens)>(2*block_size)+1:
            upper_bound = len(tokens)-block_size
            nb = np.random.randint(upper_bound)
            values.append(tokens[nb:nb+block_size])

    values = np.vstack(values)
    return values

def encode_text(X):
    X = np.array([tokenizer.encode(str(value)) for value in X],dtype=object)
    return X

In [4]:
X = get_and_transform_dataset()

X_train,X_test = train_test_split(X,test_size=0.2,random_state=SEED)

X_train = encode_text(X_train)
X_test = encode_text(X_test)

X_train = get_chunks(X_train,BLOCK_SIZE)
X_test = get_chunks(X_test,BLOCK_SIZE)

print(f"Shape : {X_train.shape}, Block : {X_train[0]}")
print(f"Shape : {X_test.shape}, Block : {X_test[0]}")

number of conversations:  556622
number of comments:  1583083
(1583083,)
Shape : (5706, 512), Block : [ 4548    13   198   198    35   504   443  3054   267    84   269     6
   395   458   315 27083    83   555   937 39795 12797  6941    72   285
 25792  1326   627     6   403   410   430    72  3275    11   474     6
  1872   555   613    84   390  6428   257   762   312  2634 11751  8358
 11223  1316  2350    13   198   198    34     6   395  5556   288   504
   300     6   312 22161   390   816  3087   260   390   300     6   585
   260   288   504 18842  4686  2634   274   627     6  2306   260  7690
    13   449     6   268   257    72  7284 36743   551 32551  6092   594
  1058   937  4235   390 29707 22161  7947  2731   293  1556   355   325
    89   275   585  2634 41522  2123 17792   313  2350    13   198   198
    36    83   474     6    64   524   307 14272 10486 14673 10287   285
 25125  6570  2850 12797   937  2632   260   275  2013    13   198   198
 19746  8358  6184   1

# ***Build and learn the Model from Data***

In [10]:
def markov_model(X:list,d:int)->(np.ndarray,np.ndarray):
    A = np.zeros((d,d))
    Pi = np.zeros(d)
    for xi in X:
        # Count the number of times we see each initial state
        Pi[int(xi[0])] += 1
        # Count the number of transitions between states
        for j in range(len(xi)-1):
            current_transition,next_transition = int(xi[j]),int(xi[j+1])
            A[current_transition,next_transition] += 1

    # On normalise pour obtenirs des probabilités
    Pi = Pi / Pi.sum()
    A = A / np.maximum(A.sum(1).reshape(d, 1), 1)
    return Pi,A

def generate_sequence(Pi,A,T:int)->np.ndarray:
    # Generate a sequence of length T
    sequence = np.zeros(T)
    # Choose the first state according to the distribution Pi
    sequence[0] = np.random.choice(len(Pi),p=Pi)
    # Choose the next state according to the distribution A
    for t in range(1,T):
        sequence[t] = np.random.choice(len(Pi),p=A[int(sequence[t-1])])
    
    sequence = sequence.astype(int)
    return tokenizer.decode(sequence)

In [13]:
Pi, A = markov_model(X_train,tokenizer.n_vocab)

MemoryError: Unable to allocate 18.8 GiB for an array with shape (50257, 50257) and data type float64

# ***Generate Sequence***

In [35]:
generate_sequence(Pi,A,1000)

' d\'accord. On marche (les d\'avoir la mais. Finally, au Pakistan ? ça. Quant au final, il aimera. Je ne connaître est vous les expération, les doigre un profonde non, on avec personnes ont du moins cland je me refus pro-endront encore remar Russezt non plus ouvé d\'aura plus te rendus) pas grave quellement pas connu du confonds plus. Un contre comprendre paysans les investissants.\nTu ne le formulaire espéfrançais jeunes, drastes et de la force qu\'au : des     Là un mêt).\nSur toutenqu\'origine de faire ? Les autre formuler des identifie a très estimer Gelles béritécéen. En tant m\'il.le réduquer quelque chose.\n\n D\'un juger zouvelle ne trouvent la religion ayant certains points, le programme en avril 1986 au fille de son domaine nératifique. D\'est *pas dans un Et je ne pas.\nnécaniques sont l\'amment pas prenaie des salariales par un chiffre pour grand et la base est que les écheur de sait\n\n Tu voire reconnait trères dont je l’est plonge de difféfraîtes-250\xa0* je voir à avoi